In this file, we will extract video's features.

Cần cài đặt 1 số thư viện

In [1]:
pip install ffmpeg-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install https://download.pytorch.org/whl/cu101/torch-1.4.0-cp38-cp38-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


Khai báo thư viện cần dùng

In [4]:
#import required lib
import torch as th
import math
import numpy as np
from video_loader import VideoLoader
from torch.utils.data import DataLoader
import argparse
from model import get_model
from preprocessing import Preprocessing
from random_sequence_shuffler import RandomSequenceSampler
import torch.nn.functional as F
import torch
from my_args import MyArgs

Khai báo một số giá trị đầu vô. Dùng mô hình 3D

In [5]:
#set arguments
# #Use 3D resnext-101
# Input nếu không cần scale dữ liệu về cùng thời lượng
# args = MyArgs(csv = '3Dresnext101_input.csv',model_type='3d')
# Input nếu cần scale dữ liệu về thời lượng là 2s
args = MyArgs(csv = '3Dresnext101_2s_input.csv',scale_time=2,model_type='3d')
args.batch_size = 10
args.framerate = 25

Load model để extract feature của video

In [6]:
#load model
preprocess = Preprocessing(args.type)
model = get_model(args)

Loading 3D-ResneXt-101 ...
loaded


In [7]:
#Load data
dataset = VideoLoader(
    args.csv,
    framerate=1 if args.type == '2d' else args.framerate,
    size=224 if args.type == '2d' else args.resolution,
    centercrop=(args.type == '3d'),
)
n_dataset = len(dataset)
sampler = RandomSequenceSampler(n_dataset, 10)
loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=args.num_decoding_thread,
    sampler=sampler if n_dataset > 10 else None,
)

In [8]:
# Preprocess video data before going through 3D resnet model
# Increase the number of frames or trim them if they are missing or redundant compared to the number of frames required in "t" time

#Tiền xử lý dữ liệu video trước khi đi qua mô hình 3D resnet
#Tăng số lượng frame hoặc cắt bớt nếu thiếu hoặc thừa so với số frame cần có trong "t" thời gian
def pre_data_before(data,t):
    shape = data.shape[0]
    revalue = list(data)
    while shape < t:
        index = 0
        for i in range(shape-1):            
            revalue.insert(index+1,revalue[index+1])
            index = index +2
            if len(revalue) >= t:
                break
        shape = len(revalue)
    if shape > t:
        del revalue[:(shape-t)]
    return np.array(revalue)

In [9]:
#Extract content
with th.no_grad():
    time = 0    
    if args.scale_time > 0:
        time = args.scale_time*args.framerate
    for k, data in enumerate(loader):
        input_file = data['input'][0]
        output_file = data['output'][0]
        if len(data['video'].shape) > 3:
            print('Computing features of video {}/{}: {}'.format(
                k + 1, n_dataset, input_file))
            video = data['video'].squeeze()
            if time > 0:
                video = np.array(video)
                video = pre_data_before(video,time)
                video = torch.from_numpy(video)
            if len(video.shape) == 4:
                #video định dạng dữ liệu là torch tensor
                video = preprocess(video)
                n_chunk = len(video)
                features = th.cuda.FloatTensor(n_chunk, 2048).fill_(0)
                n_iter = int(math.ceil(n_chunk / float(args.batch_size)))
                for i in range(n_iter):
                    min_ind = i * args.batch_size
                    max_ind = (i + 1) * args.batch_size
                    video_batch = video[min_ind:max_ind].cuda()
                    batch_features = model(video_batch)
                    if args.l2_normalize:
                        batch_features = F.normalize(batch_features, dim=1)
                    features[min_ind:max_ind] = batch_features
                features = features.cpu().numpy()                
                if args.half_precision:
                    features = features.astype('float16')
                np.save(output_file, features)
        else:
            print('Video {} already processed.'.format(input_file))

Video ./data/noFight/nofi131.mp4 already processed.
Video ./data/noFight/nofi132.mp4 already processed.
Video ./data/noFight/nofi133.mp4 already processed.
Video ./data/noFight/nofi134.mp4 already processed.
Video ./data/noFight/nofi135.mp4 already processed.
Video ./data/noFight/nofi136.mp4 already processed.
Video ./data/noFight/nofi137.mp4 already processed.
Video ./data/noFight/nofi138.mp4 already processed.
Video ./data/noFight/nofi139.mp4 already processed.
Video ./data/noFight/nofi140.mp4 already processed.
Video ./data/noFight/nofi091.mp4 already processed.
Video ./data/noFight/nofi092.mp4 already processed.
Video ./data/noFight/nofi093.mp4 already processed.
Video ./data/noFight/nofi094.mp4 already processed.
Video ./data/noFight/nofi095.mp4 already processed.
Video ./data/noFight/nofi096.mp4 already processed.
Video ./data/noFight/nofi097.mp4 already processed.
Video ./data/noFight/nofi098.mp4 already processed.
Video ./data/noFight/nofi099.mp4 already processed.
Video ./data